# What is a Metrics?

A metric is a numerical value that quantifies the quality of a model's predictions. For example, `accuracy` is a metric used to assess the performance of a classifier model. 

## How can we log metrics with MLflow? 

In this case we can use `mlflow.log_metric` or `mlflow.log_metrics`

In [1]:
from mlflow_for_ml_dev.utils.utils import get_root_project
import mlflow 

In [3]:
# create and set experiment 
experiment_name = "metrics"
experiment = mlflow.set_experiment(experiment_name)

In [4]:
with mlflow.start_run(run_name="logging metrics") as run:
    print(f"Run: {run.info.run_id}")

    # log metrics
    mlflow.log_metric("accuracy", 0.9)

    # log metrics 
    mlflow.log_metrics({"precision": 0.8, "recall": 0.7, "f1": 0.75})

Run: 22339bc513c24caea89a89b315a6391b


# Creating a custom metric

In [6]:
from mlflow.metrics import make_metric

* eval_fn –

A function that computes the metric with the following signature:
```python
def eval_fn(
    predictions: pandas.Series,
    targets: pandas.Series,
    metrics: Dict[str, MetricValue],
    **kwargs,
) -> Union[float, MetricValue]:
    """
    Args:
        predictions: A pandas Series containing the predictions made by the model.
        targets: (Optional) A pandas Series containing the corresponding labels
            for the predictions made on that input.
        metrics: (Optional) A dictionary containing the metrics calculated by the
            default evaluator.  The keys are the names of the metrics and the values
            are the metric values.  To access the MetricValue for the metrics
            calculated by the system, make sure to specify the type hint for this
            parameter as Dict[str, MetricValue].  Refer to the DefaultEvaluator
            behavior section for what metrics will be returned based on the type of
            model (i.e. classifier or regressor).  kwargs: Includes a list of args
            that are used to compute the metric. These args could information coming
            from input data, model outputs or parameters specified in the
            `evaluator_config` argument of the `mlflow.evaluate` API.
        kwargs: Includes a list of args that are used to compute the metric. These
            args could be information coming from input data, model outputs,
            other metrics, or parameters specified in the `evaluator_config`
            argument of the `mlflow.evaluate` API.

    Returns: MetricValue with per-row scores, per-row justifications, and aggregate
        results.
    """
    

```

* greater_is_better – Whether a higher value of the metric is better.

* name – The name of the metric. This argument must be specified if eval_fn is a lambda function or the eval_fn.__name__ attribute is not available.

* long_name – (Optional) The long name of the metric. For example, "mean_squared_error" for "mse".

* version – (Optional) The metric version. For example v1.

* metric_details – (Optional) A description of the metric and how it is calculated.

* metric_metadata – (Optional) A dictionary containing metadata for the metric.

* genai_metric_args – (Optional) A dictionary containing arguments specified by users when calling make_genai_metric or make_genai_metric_from_prompt. Those args are persisted so that we can deserialize the same metric object later.

In [7]:
import numpy as np

In [8]:
def squared_diff_plus_one(eval_df, _builtin_metrics):
    """
    This example custom metric function creates a metric based on the ``prediction`` and
    ``target`` columns in ``eval_df`.
    """
    return np.sum(np.abs(eval_df["prediction"] - eval_df["target"] + 1) ** 2)


In [10]:
custom_metric = make_metric(
        eval_fn=squared_diff_plus_one,
        greater_is_better=False,
    )

In [19]:
from sklearn.datasets import make_regression
from sklearn.ensemble import RandomForestRegressor
import pandas as pd

In [14]:
x, y = make_regression(n_samples=100, n_features=1, noise=0.1)

In [20]:
regressor = RandomForestRegressor()
regressor.fit(x, y)

y_pred = regressor.predict(x)

In [21]:
eval_df = pd.DataFrame({"prediction": y_pred, "target": y})
eval_df.head()

,prediction,target
0,-22.116520,-22.093833
1,92.948722,92.964142
2,-49.510976,-50.315632
3,61.065630,61.358347
4,73.184361,72.554530


In [23]:
squared_diff_plus_one(eval_df, _builtin_metrics=None)

358.3227485317362

In [26]:
custom_metric.greater_is_better

False

In [33]:
results = mlflow.evaluate(data = eval_df, extra_metrics=[custom_metric], model_type = "regressor", predictions="target", targets="prediction")

c:\Users\manue\projects\mlflow_for_ml_dev\.venv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
2024/08/15 12:32:37 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2024/08/15 12:32:37 INFO mlflow.models.evaluation.default_evaluator: Shap explainer ExactExplainer is used.
2024/08/15 12:32:37 WARNING mlflow.models.evaluation.default_evaluator: Shap evaluation failed. Reason: IndexError('list index out of range'). Set logging level to DEBUG to see the full traceback.


In [35]:
results.metrics

{'example_count': 100,
 'mean_absolute_error': 0.73900032740808,
 'mean_squared_error': 2.5245445692752297,
 'root_mean_squared_error': 1.5888815466469581,
 'sum_on_target': 2163.4540178832294,
 'mean_on_target': 21.634540178832296,
 'r2_score': 0.9996189126086862,
 'max_error': 10.206270195868342,
 'mean_absolute_percentage_error': 0.018866046976628085,
 'squared_diff_plus_one': 346.5861653233097}